In [1]:
!pip install tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\vamsi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow-addons

                                              0.0/719.8 kB ? eta -:--:--
     -----                                  112.6/719.8 kB 6.8 MB/s eta 0:00:01
     ---------------------                  399.4/719.8 kB 5.0 MB/s eta 0:00:01
     -------------------------------------- 719.8/719.8 kB 6.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\vamsi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install imageio

                                              0.0/313.0 kB ? eta -:--:--
     -----------------------                194.6/313.0 kB 4.0 MB/s eta 0:00:01
     -------------------------------------- 313.0/313.0 kB 4.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\vamsi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
!pip install librosa

                                              0.0/253.7 kB ? eta -:--:--
     ------------------------------         204.8/253.7 kB 4.1 MB/s eta 0:00:01
     -------------------------------------- 253.7/253.7 kB 5.2 MB/s eta 0:00:00
                                              0.0/377.0 kB ? eta -:--:--
     -----------------------------------   358.4/377.0 kB 10.9 MB/s eta 0:00:01
     -------------------------------------- 377.0/377.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/9.2 MB ? eta -:--:--
     -                                        0.4/9.2 MB 8.9 MB/s eta 0:00:01
     ---                                      0.8/9.2 MB 8.6 MB/s eta 0:00:01
     -----                                    1.2/9.2 MB 8.7 MB/s eta 0:00:01
     -------                                  1.6/9.2 MB 8.7 MB/s eta 0:00:01
     --------                                 2.0


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\vamsi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
!pip install Ipython


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\vamsi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
import librosa

import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import IPython.display as ipd
from IPython.display import clear_output

from tqdm.notebook import tqdm

import glob
import imageio
import time

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [14]:
seed=111
tf.compat.v1.set_random_seed(seed)

session_config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
session = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_config)

tf.compat.v1.keras.backend.set_session(session)

In [20]:
# Seperating the training and testing data
# Dataset already contains an 75-25 split of training and testing (check the csv file for details)

# Read the CSV file
df = pd.read_csv('data\warblrb10k_public_metadata.csv')

# Create lists for train and test datasets
train_files = [] # Contains paths of all files needed for training
test_files = []  # Contains paths of all files needed for testing

# Iterate over the DataFrame
for index, row in tqdm(df.iterrows()):
    itemid = row['itemid']
    hasbird = row['hasbird']
    
    # Define the path of the audio file
    file_path = os.path.join('data\wav', f'{itemid}.wav')  # replace .wav with your audio file extension
    
    # If hasbird is 1, add the file to the train list
    if hasbird == 1:
        train_files.append(file_path)
    # Else, add the file to the test list
    else:
        test_files.append(file_path)

print(len(train_files))
print(len(test_files))

0it [00:00, ?it/s]

6045
1955


6045
1955
